## TAX BENEFITS OF OPPORTUNITY ZONES- PREDICTING SALE PRICE IN AMES IOWA

## Table of Contents

- [Problem Statement](#Problem-Statemet)
- [Executive Summary](#Executive-Summary)
- [Data Dictionary](#Data-Dictionary)
- [Loading Data](#Loading-Data)
- [Data Cleaning](#Data-Cleaning)
- [EDA](#EDA)
- [Feature Engineering](#Feature-Engineering)
- [Model Preparation](#Model-Preparation)
- [Model Selection](#Model-Selection)
- [Conclusion](#Model-Selection)
- [Recommendations](#Reccomendations)
- [Resources](#Resources)



# Problem Statement

TNA / ATA = β0 + β1(1/ATA) + β2(ΔSales – ΔRec / ATA) + β3(GPPE / ATA ) + ε
TNA= Total net accruals 
NOA= Net operating accruals
ATA = Average total assets
ΔSales = Change in sales
ΔRec= Change in accounts receivable
GPPE = Gross PP&E

During election time, we see the amount of restatements on accruals go up for company, Accrual accounting is required by GAAP and companies have to spend loads of money to adapt to these standards. I want to create a linear model to predict the amount of accruals for a company, so we can plan the audit process accordingly. I want to use R2 as my metric to deermine  Jones Model is used to prodect the total net accruals for a company. This model is going to input variables from 2016 Q1-Q4 data into the regression model. After We create the model, we are going to train the model on a prior period data. Then we are going to use the model to test it on the next period(2017 Q3).

# Executive Summary

My approach was to first decide what platforms have this data available. I found that the data is readily avaliable on Kaggle.From initial data review, I hypothesized that the 'Overall Qual', 'TotalSqFt', 'Gr Liv Area', 'Exter Qual', 'Kitchen Qual' variables would have the most influence over sales price. The abundance of variables, forced me to dive deeper into the data in order to find the most optimal combinations in order to find the highest scoring model. We want a high majority of the data to be explained by our model. 


EDA helped me not only set up a preprocessing plan for our model, but helped me understand the health of the data One example was that “Total Square feet” showed up high on the correlation heat map, meaning that this variable will be useful in predicting sale price. For preprocessing, I chose to fill all null values wiht na, assuming that those values actually reperesented none of that type. I also created feature engineering for square feet, bathrooms, and porch square feet in order to simplify the model, so I could use those variables in the model.  For modeling, I chose to use linear regression and lasso regression because of the way it focuses on the relationship of features and our predicted variable(sale price). 

|Model|Type|Train R2|Testing R2|
|---|---|---|---|
|**Model 1**|*Linear*|.84|.84| 
|**Model 2**|*Linear*|.93|.94|
|**Model 3**|*Ridge*|.91|.88|
|**Model 4**|*Lasso*|.91|.88|

My approach to picking the best model, is by assessing the R2 Score. At first, I tried fitting a model with all variables, but decided that is overkill for our clients. I simplified the model by choosing the features with the highest correlations in predicting sale price. I will use Lasso regression to identify correlations close to 0, and remove those from the model. I will asses performance based on R^2 

## Data Dictionary

Refer to Kaggle for Data Description

- [Kaggle Data](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt)

## Loading Data

In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn import linear_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

## Jones Model

## 

In [4]:
num =  pd.read_csv('./2017q2/num.txt', sep = '\t', encoding="latin-1")
tag =  pd.read_csv('./2017q2/tag.txt', sep = '\t', encoding="latin-1")
pre =  pd.read_csv('./2017q2/pre.txt', sep = '\t', encoding="latin-1")
sub =  pd.read_csv('./2017q2/sub.txt', sep = '\t', encoding="latin-1")

### merge data

In [5]:
firstmerge = pd.merge(pre, sub, how = "left", on=['adsh'])   #left merge

secondmerge = pd.merge(firstmerge, tag, how= "left", on =['tag', 'version'])

thirdmerge = pd.merge(secondmerge, num, how = "left", on =['adsh', 'tag', 'version'])
sec_table= thirdmerge[['cik','adsh' , 'afs' , 'qtrs' , 'fy' , 'fp', 'tag', 'version', 'value' , 'ddate']].sort_values(['cik'])
sec_table.head()

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
513461,1800,0001104659-17-029430,1-LAF,1.0,2017.0,Q1,ComprehensiveIncomeNetOfTax,us-gaap/2016,1.230000e+08,20160331.0
1291465,1800,0001104659-17-029430,1-LAF,1.0,2017.0,Q1,PaymentsForRepurchaseOfCommonStock,us-gaap/2016,5.190000e+08,20160331.0
1291466,1800,0001104659-17-029430,1-LAF,1.0,2017.0,Q1,PaymentsForRepurchaseOfCommonStock,us-gaap/2016,9.500000e+07,20170331.0
440797,1800,0001104659-17-029430,1-LAF,0.0,2017.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,8.706000e+09,20170331.0
440796,1800,0001104659-17-029430,1-LAF,0.0,2017.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,1.862000e+10,20161231.0


### find unique tags

In [6]:
tags =sec_table.tag.unique().tolist()
tags

['ComprehensiveIncomeNetOfTax',
 'PaymentsForRepurchaseOfCommonStock',
 'CashAndCashEquivalentsAtCarryingValue',
 'OtherComprehensiveIncomeLossPensionAndOtherPostretirementBenefitPlansTax',
 'EarningsPerShareBasicAbstract',
 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterestAbstract',
 'NetCashProvidedByUsedInInvestingActivitiesAbstract',
 'ProceedsFromSalesOfBusinessAffiliateAndProductiveAssets',
 'IncreaseDecreaseInInventories',
 'EffectOfExchangeRateOnCashAndCashEquivalents',
 'AmortizationOfIntangibleAssets',
 'PropertyPlantAndEquipmentNet',
 'LiabilitiesCurrentAbstract',
 'AccruedIncomeTaxesCurrent',
 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
 'IncreaseDecreaseInOtherOperatingCapitalNet',
 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'OtherComprehensiveIncomeAvailableforsaleSecuritiesAdjustmentNetOfTaxPortionAttributableToParent',
 'OtherComprehensiveIncomeForeignCurrencyTransact

## Sort by tags 

In [7]:
Regression_2016 =  sec_table[(sec_table.tag == 'NetIncomeLoss')
                        | (sec_table.tag == 'NetCashProvidedByUsedInOperatingActivities')
                        | (sec_table.tag == 'SalesRevenueNet')
                        | (sec_table.tag == 'Assets')
                        | (sec_table.tag == 'AccountsReceivableNetCurrent')
                        | (sec_table.tag == 'PropertyPlantAndEquipmentNet')]

### Make pivot table by quarters

In [8]:
Regression_2016.pivot_table(values='value', index=['tag', 'adsh','cik','ddate'], columns=['qtrs'])

qtrs                                                                         0.0    \
tag                          adsh                 cik     ddate                      
AccountsReceivableNetCurrent 0000002178-17-000038 2178    20161231.0  8.716200e+07   
                                                          20170331.0  8.912200e+07   
                             0000002488-17-000107 2488    20161231.0  3.110000e+08   
                                                          20170331.0  4.940000e+08   
                             0000002969-17-000019 2969    20160930.0  1.146200e+09   
...                                                                            ...   
SalesRevenueNet              0001640334-17-001149 1407704 20161231.0           NaN   
                             0001663577-17-000163 1670869 20160331.0           NaN   
                                                          20170331.0           NaN   
                             0001391609-17-000136 1223533 20160331.0           NaN   
                                                          20170331.0           NaN   

qtrs                                                                     1.0    \
tag                          adsh                 cik     ddate                  
AccountsReceivableNetCurrent 0000002178-17-000038 2178    20161231.0       NaN   
                                                          20170331.0       NaN   
                             0000002488-17-000107 2488    20161231.0       NaN   
                                                          20170331.0       NaN   
                             0000002969-17-000019 2969    20160930.0       NaN   
...                                                                        ...   
SalesRevenueNet              0001640334-17-001149 1407704 20161231.0       NaN   
                             0001663577-17-000163 1670869 20160331.0  157138.0   
                                                          20170331.0  117814.0   
                             0001391609-17-000136 1223533 20160331.0       0.0   
                                                          20170331.0       0.0   

qtrs                                                                  2.0    \
tag                          adsh                 cik     ddate               
AccountsReceivableNetCurrent 0000002178-17-000038 2178    20161231.0    NaN   
                                                          20170331.0    NaN   
                             0000002488-17-000107 2488    20161231.0    NaN   
                                                          20170331.0    NaN   
                             0000002969-17-000019 2969    20160930.0    NaN   
...                                                                     ...   
SalesRevenueNet              0001640334-17-001149 1407704 20161231.0    NaN   
                             0001663577-17-000163 1670869 20160331.0    NaN   
                                                          20170331.0    NaN   
                             0001391609-17-000136 1223533 20160331.0    NaN   
                                                          20170331.0    NaN   

qtrs                                                                     3.0    \
tag                          adsh                 cik     ddate                  
AccountsReceivableNetCurrent 0000002178-17-000038 2178    20161231.0       NaN   
                                                          20170331.0       NaN   
                             0000002488-17-000107 2488    20161231.0       NaN   
                                                          20170331.0       NaN   
                             0000002969-17-000019 2969    20160930.0       NaN   
...                                                                        ...   
SalesRevenueNet              0001640334-17-001149 1407704 20161231.0       NaN   
                             0001663577-17-00016

### Unique Quarters

In [9]:
Regression_2016.qtrs.unique()

array([  0.,   1.,   4.,   3.,   2.,  nan,  27.,  52.,   8.,  25.,  74.,
       158., 159., 100., 101., 102., 104.,  98.,  66.,  47.,  23.,  45.,
        43.,  44.,  42.,  58.,  37.,  33.,  38.,  35.,  34.,  28.,   7.,
        31.,  30.,  16.,  18.,  17.,  19.,  13.,  14.,  15.,  11.,   5.,
        10.,  12.,   6.])

# FILTER OUT BY DDATE QUARTER 4 Data

In [10]:
qrtr4index = Regression_2016[(Regression_2016.qtrs == 4.) | (Regression_2016.qtrs == 1.)]
qrtr4 = qrtr4index.pivot_table(values='value', index=['cik','ddate'], columns=['tag'], dropna=True)
qrtr4

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
1800    20160331.0                                 -93000000.0    316000000.0   
        20170331.0                                 574000000.0    419000000.0   
1961    20151231.0                                   -480365.0     -4037365.0   
        20160331.0                                   -197876.0      -137335.0   
        20161231.0                                   -681364.0     -1132906.0   
...                                                        ...            ...   
1700175 20160331.0                                         NaN        80000.0   
        20170331.0                                         NaN       107000.0   
1702494 20160331.0                                 166699000.0            NaN   
        20170331.0                                  58479000.0            NaN   
1708304 20170331.0                                    -20837.0       -28838.0   

tag                 SalesRevenueNet  
cik     ddate                        
1800    20160331.0     4.885000e+09  
        20170331.0     6.335000e+09  
1961    20151231.0              NaN  
        20160331.0              NaN  
        20161231.0              NaN  
...                             ...  
1700175 20160331.0              NaN  
        20170331.0              NaN  
1702494 20160331.0              NaN  
        20170331.0              NaN  
1708304 20170331.0              NaN  

[15257 rows x 3 columns]

filter out by those dates

In [11]:
data_setQ4 = Regression_2016[(Regression_2016.ddate == 20161231)
                   | (Regression_2016.ddate == 20151231)]

### pivot table by data

In [12]:
data_tableQ4 = pd.pivot_table(data_setQ4,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ4

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
1800    20161231.0                  3.248000e+09  5.266600e+10   
1961    20151231.0                           NaN  2.629800e+04   
        20161231.0                           NaN  9.337800e+04   
2098    20161231.0                  2.002100e+07  9.206600e+07   
2178    20161231.0                  8.716200e+07  2.468720e+08   
...                                          ...           ...   
1698519 20161231.0                           NaN  1.749000e+03   
1699018 20161231.0                           NaN  1.159730e+08   
1700175 20161231.0                           NaN  9.801500e+07   
1702494 20161231.0                  7.793800e+07  5.349795e+09   
1708304 20161231.0                           NaN  2.745000e+03   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
1800    20161231.0                                         NaN            NaN   
1961    20151231.0                                   -480365.0     -4037365.0   
        20161231.0                                   -681364.0     -1132906.0   
2098    20161231.0                                         NaN            NaN   
2178    20161231.0                                         NaN            NaN   
...                                                        ...            ...   
1698519 20161231.0                                         NaN            NaN   
1699018 20161231.0                                         NaN            NaN   
1700175 20161231.0                                         NaN            NaN   
1702494 20161231.0                                         NaN            NaN   
1708304 20161231.0                                     -4670.0        -4670.0   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  
cik     ddate                                                      
1800    20161231.0                  5.705000e+09              NaN  
1961    20151231.0                           NaN              NaN  
        20161231.0                           NaN              NaN  
2098    20161231.0                  7.973000e+06              NaN  
2178    20161231.0                  4.632500e+07              NaN  
...                                          ...              ...  
1698519 20161231.0                           NaN              NaN  
1699018 20161231.0                  4.340000e+06              NaN  
1700175 20161231.0                  3.716000e+06              NaN  
1702494 20161231.0                  4.954619e+09              NaN  
1708304 20161231.0                           NaN              NaN  

[5769 rows x 6 columns]

create engineer accruals column

In [13]:
data_tableQ4['TOTALNETACCRUALS'] =  data_tableQ4['NetIncomeLoss'] - data_tableQ4['NetCashProvidedByUsedInOperatingActivities']
data_tableQ4

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
1800    20161231.0                  3.248000e+09  5.266600e+10   
1961    20151231.0                           NaN  2.629800e+04   
        20161231.0                           NaN  9.337800e+04   
2098    20161231.0                  2.002100e+07  9.206600e+07   
2178    20161231.0                  8.716200e+07  2.468720e+08   
...                                          ...           ...   
1698519 20161231.0                           NaN  1.749000e+03   
1699018 20161231.0                           NaN  1.159730e+08   
1700175 20161231.0                           NaN  9.801500e+07   
1702494 20161231.0                  7.793800e+07  5.349795e+09   
1708304 20161231.0                           NaN  2.745000e+03   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
1800    20161231.0                                         NaN            NaN   
1961    20151231.0                                   -480365.0     -4037365.0   
        20161231.0                                   -681364.0     -1132906.0   
2098    20161231.0                                         NaN            NaN   
2178    20161231.0                                         NaN            NaN   
...                                                        ...            ...   
1698519 20161231.0                                         NaN            NaN   
1699018 20161231.0                                         NaN            NaN   
1700175 20161231.0                                         NaN            NaN   
1702494 20161231.0                                         NaN            NaN   
1708304 20161231.0                                     -4670.0        -4670.0   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  \
cik     ddate                                                       
1800    20161231.0                  5.705000e+09              NaN   
1961    20151231.0                           NaN              NaN   
        20161231.0                           NaN              NaN   
2098    20161231.0                  7.973000e+06              NaN   
2178    20161231.0                  4.632500e+07              NaN   
...                                          ...              ...   
1698519 20161231.0                           NaN              NaN   
1699018 20161231.0                  4.340000e+06              NaN   
1700175 20161231.0                  3.716000e+06              NaN   
1702494 20161231.0                  4.954619e+09              NaN   
1708304 20161231.0                           NaN              NaN   

tag                 TOTALNETACCRUALS  
cik     ddate                         
1800    20161231.0               NaN  
1961    20151231.0        -3557000.0  
        20161231.0         -451542.0  
2098    20161231.0               NaN  
2178    20161231.0               NaN  
...                              ...  
1698519 20161231.0               NaN  
1699018 20161231.0               NaN  
1700175 20161231.0               NaN  
1702494 20161231.0               NaN  
1708304 20161231.0               0.0  

[5769 rows x 7 columns]

drop columns

In [14]:
Q4Regres = data_tableQ4.dropna(axis = 0)
Q4Regres

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
18172   20151231.0                  7.319100e+07  4.935060e+08   
        20161231.0                  6.438500e+07  3.293280e+08   
75439   20151231.0                  1.798000e+06  1.837800e+07   
        20161231.0                  1.346000e+06  1.777000e+07   
313616  20151231.0                  2.985100e+09  4.822220e+10   
...                                          ...           ...   
1620702 20161231.0                  3.759600e+07  2.448279e+09   
1628871 20151231.0                  8.559630e+05  4.862328e+06   
        20161231.0                  1.415083e+06  9.635460e+06   
1629665 20151231.0                  4.541180e+05  1.669539e+07   
        20161231.0                  4.553830e+05  2.088230e+07   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
18172   20151231.0                               -2.213300e+07  -1.651670e+08   
        20161231.0                               -3.496200e+07  -6.891900e+07   
75439   20151231.0                               -3.880000e+05  -8.590000e+05   
        20161231.0                               -1.091000e+06  -2.163000e+06   
313616  20151231.0                                3.801800e+09   2.023000e+09   
...                                                        ...            ...   
1620702 20161231.0                               -1.423700e+07  -5.303200e+07   
1628871 20151231.0                               -2.634330e+06  -7.525821e+06   
        20161231.0                               -1.484273e+06  -3.126072e+06   
1629665 20151231.0                               -3.746217e+06  -3.011543e+06   
        20161231.0                                5.273598e+06  -5.626460e+05   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  \
cik     ddate                                                       
18172   20151231.0                  6.456100e+07     3.852175e+08   
        20161231.0                  4.971000e+07     3.234335e+08   
75439   20151231.0                  1.941000e+06     9.887000e+06   
        20161231.0                  1.557000e+06     9.238000e+06   
313616  20151231.0                  2.302700e+09     9.378350e+09   
...                                          ...              ...   
1620702 20161231.0                  1.355362e+09     1.347315e+08   
1628871 20151231.0                  1.173980e+05     1.193343e+07   
        20161231.0                  5.934907e+06     1.877596e+07   
1629665 20151231.0                  5.185217e+06     1.120999e+06   
        20161231.0                  1.589709e+07     3.055882e+06   

tag                 TOTALNETACCRUALS  
cik     ddate                         
18172   20151231.0     -1.430340e+08  
        20161231.0     -3.395700e+07  
75439   20151231.0     -4.710000e+05  
        20161231.0     -1.072000e+06  
313616  20151231.0     -1.778800e+09  
...                              ...  
1620702 20161231.0     -3.879500e+07  
1628871 20151231.0     -4.891491e+06  
        20161231.0     -1.641799e+06  
1629665 20151231.0      7.346740e+05  
        20161231.0     -5.836244e+06  

[115 rows x 7 columns]

# TIMESERIES LAGGED VAIRABLES for variables at the beggining and the end of the year

In [15]:
Q4Regres['ASSETSLAG']=Q4Regres.groupby('cik').Assets.shift()
Q4Regres['REVLAG'] = Q4Regres.groupby('cik').SalesRevenueNet.shift()
Q4Regres['ARLAG'] = Q4Regres.groupby('cik').AccountsReceivableNetCurrent.shift()
Q4Regres['REVCHANGE'] = Q4Regres['SalesRevenueNet'] - Q4Regres['REVLAG']
Q4Regres['ARCHANGE'] = Q4Regres['AccountsReceivableNetCurrent'] - Q4Regres['ARLAG']
Q4Regres['ROA'] = Q4Regres['NetIncomeLoss'] / Q4Regres['Assets']
Regres_Q4 = Q4Regres.dropna(axis = 0)

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [29]:
Regres_Q4

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
18172,20161231.0,6.438500e+07,3.293280e+08,-3.496200e+07,-6.891900e+07,4.971000e+07,3.234335e+08,-3.395700e+07,4.935060e+08,3.852175e+08,7.319100e+07,-6.178400e+07,-8.806000e+06,-0.209272
75439,20161231.0,1.346000e+06,1.777000e+07,-1.091000e+06,-2.163000e+06,1.557000e+06,9.238000e+06,-1.072000e+06,1.837800e+07,9.887000e+06,1.798000e+06,-6.490000e+05,-4.520000e+05,-0.121722
313616,20161231.0,3.186100e+09,4.529530e+10,3.521800e+09,1.650350e+09,2.354000e+09,1.073335e+10,-1.871450e+09,4.822220e+10,9.378350e+09,2.985100e+09,1.355000e+09,2.010000e+08,0.036435
708821,20161231.0,3.070500e+07,1.245110e+08,1.099600e+07,1.783000e+06,7.035000e+06,2.296530e+08,-9.213000e+06,1.162510e+08,2.290030e+08,2.953000e+07,6.500000e+05,1.175000e+06,0.014320
946454,20161231.0,1.631600e+07,6.243900e+07,-3.462000e+06,-4.080000e+05,1.169500e+07,7.625400e+07,3.054000e+06,6.686400e+07,7.805900e+07,1.103200e+07,-1.805000e+06,5.284000e+06,-0.006534
949428,20161231.0,5.398000e+06,2.483000e+07,-7.004000e+06,-2.347400e+07,1.158000e+06,3.683900e+07,-1.647000e+07,2.510200e+07,3.973800e+07,4.255000e+06,-2.899000e+06,1.143000e+06,-0.945389
1004673,20161231.0,1.280000e+05,3.942800e+07,-3.585000e+06,-9.735000e+06,9.150000e+05,1.959000e+06,-6.150000e+06,4.528500e+07,2.641000e+06,7.240000e+05,-6.820000e+05,-5.960000e+05,-0.246906
1011395,20161231.0,6.580000e+04,3.176360e+05,-9.894640e+05,-1.191474e+06,4.473900e+04,7.696710e+05,-2.020100e+05,3.101450e+05,8.970000e+04,4.410000e+04,6.799710e+05,2.170000e+04,-3.751067
1043894,20161231.0,1.452000e+03,1.403650e+05,-3.274000e+04,-5.357930e+05,1.101700e+04,4.960880e+05,-5.030530e+05,3.073030e+05,9.329430e+05,9.427700e+04,-4.368550e+05,-9.282500e+04,-3.817141


Quarter 3

In [16]:
qrtr3index = Regression_2016[(Regression_2016.qtrs == 3.0)]
qrtr3 = qrtr3index.pivot_table(values='value', index=['cik','ddate'], columns=['tag'], dropna=True)
qrtr3

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
2034    20160331.0                                         NaN     27992000.0   
        20170331.0                                         NaN      9409000.0   
5656    20160229.0                                    379035.0       318885.0   
        20170228.0                                   -309672.0      -392946.0   
6284    20160430.0                                         NaN      3361000.0   
...                                                        ...            ...   
1683252 20170331.0                                    -22492.0       -22492.0   
1685766 20170228.0                                         NaN        -9915.0   
1687065 20170531.0                                    -17694.0       -20593.0   
1691337 20160331.0                                         NaN      3082000.0   
        20170331.0                                         NaN      5019000.0   

tag                 SalesRevenueNet  
cik     ddate                        
2034    20160331.0      423100000.0  
        20170331.0      443698000.0  
5656    20160229.0              NaN  
        20170228.0          48076.0  
6284    20160430.0              NaN  
...                             ...  
1683252 20170331.0              NaN  
1685766 20170228.0              NaN  
1687065 20170531.0              NaN  
1691337 20160331.0              NaN  
        20170331.0              NaN  

[1096 rows x 3 columns]

In [17]:
data_setQ3 = Regression_2016[(Regression_2016.ddate == 20160930)
                   | (Regression_2016.ddate == 20150930)]

In [18]:
data_tableQ3 = pd.pivot_table(data_setQ3,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ3

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
2969    20150930.0                  1.126400e+09  1.731660e+10   
        20160930.0                  1.146200e+09  1.802860e+10   
3545    20160930.0                  4.740000e+06  4.554450e+08   
4127    20160930.0                           NaN  3.855400e+09   
8411    20160930.0                  1.130910e+08  4.539792e+09   
...                                          ...           ...   
1696195 20150930.0                           NaN           NaN   
        20160930.0                           NaN  1.375500e+04   
1697834 20160930.0                           NaN  2.050860e+06   
1698022 20150930.0                           NaN           NaN   
        20160930.0                           NaN           NaN   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
2969    20150930.0                                         NaN    811200000.0   
        20160930.0                                         NaN    512550000.0   
3545    20160930.0                                         NaN            NaN   
4127    20160930.0                                         NaN            NaN   
8411    20160930.0                                         NaN            NaN   
...                                                        ...            ...   
1696195 20150930.0                                         NaN        -3714.0   
        20160930.0                                         NaN        -3500.0   
1697834 20160930.0                                         NaN            NaN   
1698022 20150930.0                                         NaN      1236261.0   
        20160930.0                                         NaN      1040372.0   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  
cik     ddate                                                      
2969    20150930.0                  8.237300e+09     4.893950e+09  
        20160930.0                  8.259700e+09     4.724600e+09  
3545    20160930.0                  3.792470e+08              NaN  
4127    20160930.0                  8.063000e+08              NaN  
8411    20160930.0                  4.127696e+09              NaN  
...                                          ...              ...  
1696195 20150930.0                           NaN              NaN  
        20160930.0                  3.300000e+02              NaN  
1697834 20160930.0                           NaN              NaN  
1698022 20150930.0                           NaN              NaN  
        20160930.0                           NaN              NaN  

[712 rows x 6 columns]

In [19]:
data_tableQ3['TOTALNETACCRUALS'] =  data_tableQ3['NetIncomeLoss'] - data_tableQ3['NetCashProvidedByUsedInOperatingActivities']
Q3_Regression = data_tableQ3.dropna(axis = 0)
Q3_Regression['ASSETSLAG']=Q3_Regression.groupby('cik').Assets.shift()
Q3_Regression['REVLAG'] = Q3_Regression.groupby('cik').SalesRevenueNet.shift()
Q3_Regression['ARLAG'] = Q3_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q3_Regression['REVCHANGE'] = Q3_Regression['SalesRevenueNet'] - Q3_Regression['REVLAG']
Q3_Regression['ARCHANGE'] = Q3_Regression['AccountsReceivableNetCurrent'] - Q3_Regression['ARLAG']
Q3_Regression['ROA'] = Q3_Regression['NetIncomeLoss'] / Q3_Regression['Assets']
Regression_Q3 = Q3_Regression.dropna(axis = 0)

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be 

Q2

In [21]:
qrtr2index = Regression_2016[(Regression_2016.qtrs == 0.) | (Regression_2016.qtrs == 1.)]
qrtr2 = qrtr2index.pivot_table(values='value', index=['cik','ddate'], columns=['tag'], dropna=True)
qrtr2

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
1800    20160331.0                           NaN           NaN   
        20161231.0                  3.248000e+09  5.266600e+10   
        20170331.0                  4.510000e+09  7.088700e+10   
1961    20151231.0                           NaN  2.629800e+04   
        20160331.0                           NaN           NaN   
...                                          ...           ...   
1702494 20161231.0                  7.793800e+07  5.349795e+09   
        20170331.0                  1.169280e+08  5.641189e+09   
1705126 20170228.0                           NaN  2.628000e+03   
1708304 20161231.0                           NaN  2.745000e+03   
        20170331.0                           NaN  3.174140e+05   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
1800    20160331.0                                 -93000000.0    316000000.0   
        20161231.0                                         NaN            NaN   
        20170331.0                                 574000000.0    419000000.0   
1961    20151231.0                                         NaN            NaN   
        20160331.0                                   -197876.0      -137335.0   
...                                                        ...            ...   
1702494 20161231.0                                         NaN            NaN   
        20170331.0                                  58479000.0            NaN   
1705126 20170228.0                                     -1372.0        -1372.0   
1708304 20161231.0                                     -4670.0        -4670.0   
        20170331.0                                    -20837.0       -28838.0   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  
cik     ddate                                                      
1800    20160331.0                           NaN     4.885000e+09  
        20161231.0                  5.705000e+09              NaN  
        20170331.0                  7.265000e+09     6.335000e+09  
1961    20151231.0                           NaN              NaN  
        20160331.0                           NaN              NaN  
...                                          ...              ...  
1702494 20161231.0                  4.954619e+09              NaN  
        20170331.0                  5.228935e+09              NaN  
1705126 20170228.0                           NaN              NaN  
1708304 20161231.0                           NaN              NaN  
        20170331.0                  2.929150e+05              NaN  

[20041 rows x 6 columns]

In [22]:
data_setQ2 = Regression_2016[(Regression_2016.ddate == 20160630)
                   | (Regression_2016.ddate == 20150630)]
data_tableQ2 = pd.pivot_table(data_setQ2,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ2['TOTALNETACCRUALS'] =  data_tableQ2['NetIncomeLoss'] - data_tableQ2['NetCashProvidedByUsedInOperatingActivities']
Q2_Regression = data_tableQ2.dropna(axis = 0)
Q2_Regression['ASSETSLAG']=Q2_Regression.groupby('cik').Assets.shift()
Q2_Regression['REVLAG'] = Q2_Regression.groupby('cik').SalesRevenueNet.shift()
Q2_Regression['ARLAG'] = Q2_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q2_Regression['REVCHANGE'] = Q2_Regression['SalesRevenueNet'] - Q3_Regression['REVLAG']
Q2_Regression['ARCHANGE'] = Q2_Regression['AccountsReceivableNetCurrent'] - Q3_Regression['ARLAG']
Q2_Regression['ROA'] = Q2_Regression['NetIncomeLoss'] / Q3_Regression['Assets']
Regression_Q2 = Q2_Regression.dropna(axis = 0)


/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on 

In [32]:
Regression_Q2

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,


Quarter1

In [23]:
qrtr1index = Regression_2016[(Regression_2016.qtrs == 0.) | (Regression_2016.qtrs == 1.)]
qrtr1 = qrtr1index.pivot_table(values='value', index=['cik','ddate'], columns=['tag'], dropna=True)
qrtr1

tag                 AccountsReceivableNetCurrent        Assets  \
cik     ddate                                                    
1800    20160331.0                           NaN           NaN   
        20161231.0                  3.248000e+09  5.266600e+10   
        20170331.0                  4.510000e+09  7.088700e+10   
1961    20151231.0                           NaN  2.629800e+04   
        20160331.0                           NaN           NaN   
...                                          ...           ...   
1702494 20161231.0                  7.793800e+07  5.349795e+09   
        20170331.0                  1.169280e+08  5.641189e+09   
1705126 20170228.0                           NaN  2.628000e+03   
1708304 20161231.0                           NaN  2.745000e+03   
        20170331.0                           NaN  3.174140e+05   

tag                 NetCashProvidedByUsedInOperatingActivities  NetIncomeLoss  \
cik     ddate                                                                   
1800    20160331.0                                 -93000000.0    316000000.0   
        20161231.0                                         NaN            NaN   
        20170331.0                                 574000000.0    419000000.0   
1961    20151231.0                                         NaN            NaN   
        20160331.0                                   -197876.0      -137335.0   
...                                                        ...            ...   
1702494 20161231.0                                         NaN            NaN   
        20170331.0                                  58479000.0            NaN   
1705126 20170228.0                                     -1372.0        -1372.0   
1708304 20161231.0                                     -4670.0        -4670.0   
        20170331.0                                    -20837.0       -28838.0   

tag                 PropertyPlantAndEquipmentNet  SalesRevenueNet  
cik     ddate                                                      
1800    20160331.0                           NaN     4.885000e+09  
        20161231.0                  5.705000e+09              NaN  
        20170331.0                  7.265000e+09     6.335000e+09  
1961    20151231.0                           NaN              NaN  
        20160331.0                           NaN              NaN  
...                                          ...              ...  
1702494 20161231.0                  4.954619e+09              NaN  
        20170331.0                  5.228935e+09              NaN  
1705126 20170228.0                           NaN              NaN  
1708304 20161231.0                           NaN              NaN  
        20170331.0                  2.929150e+05              NaN  

[20041 rows x 6 columns]

In [24]:
data_setQ1 = Regression_2016[(Regression_2016.ddate == 20160331)
                   | (Regression_2016.ddate == 20150331)]
data_tableQ1 = pd.pivot_table(data_setQ1,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ1['TOTALNETACCRUALS'] =  data_tableQ1['NetIncomeLoss'] - data_tableQ1['NetCashProvidedByUsedInOperatingActivities']
Q1_Regression = data_tableQ1.dropna(axis = 0)
Q1_Regression['ASSETSLAG']=Q1_Regression.groupby('cik').Assets.shift()
Q1_Regression['REVLAG'] = Q1_Regression.groupby('cik').SalesRevenueNet.shift()
Q1_Regression['ARLAG'] = Q1_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q1_Regression['REVCHANGE'] = Q1_Regression['SalesRevenueNet'] - Q1_Regression['REVLAG']
Q1_Regression['ARCHANGE'] = Q1_Regression['AccountsReceivableNetCurrent'] - Q1_Regression['ARLAG']
Q1_Regression['ROA'] = Q1_Regression['NetIncomeLoss'] / Q1_Regression['Assets']
Regression_Q1 = Q1_Regression.dropna(axis = 0)

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on 

In [33]:
Regression_Q1

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
1516479,20160331.0,20663000.0,326027000.0,12315000.0,2290000.0,39618000.0,60421000.0,-10025000.0,248185000.0,67692000.0,31617000.0,-7271000.0,-10954000.0,0.007024


In [25]:
first_q = pd.merge(Regression_Q1, Regression_Q2,
                 how = 'outer')
second_q = pd.merge(first_q, Regression_Q3,
                 how = 'outer')
third_q = pd.merge(second_q, Regres_Q4,
                 how = 'outer')
third_q

tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
0,2.066300e+07,3.260270e+08,1.231500e+07,2.290000e+06,3.961800e+07,6.042100e+07,-1.002500e+07,2.481850e+08,6.769200e+07,3.161700e+07,-7.271000e+06,-1.095400e+07,0.007024
1,6.438500e+07,3.293280e+08,-3.496200e+07,-6.891900e+07,4.971000e+07,3.234335e+08,-3.395700e+07,4.935060e+08,3.852175e+08,7.319100e+07,-6.178400e+07,-8.806000e+06,-0.209272
2,1.346000e+06,1.777000e+07,-1.091000e+06,-2.163000e+06,1.557000e+06,9.238000e+06,-1.072000e+06,1.837800e+07,9.887000e+06,1.798000e+06,-6.490000e+05,-4.520000e+05,-0.121722
3,3.186100e+09,4.529530e+10,3.521800e+09,1.650350e+09,2.354000e+09,1.073335e+10,-1.871450e+09,4.822220e+10,9.378350e+09,2.985100e+09,1.355000e+09,2.010000e+08,0.036435
4,3.070500e+07,1.245110e+08,1.099600e+07,1.783000e+06,7.035000e+06,2.296530e+08,-9.213000e+06,1.162510e+08,2.290030e+08,2.953000e+07,6.500000e+05,1.175000e+06,0.014320
5,1.631600e+07,6.243900e+07,-3.462000e+06,-4.080000e+05,1.169500e+07,7.625400e+07,3.054000e+06,6.686400e+07,7.805900e+07,1.103200e+07,-1.805000e+06,5.284000e+06,-0.006534
6,5.398000e+06,2.483000e+07,-7.004000e+06,-2.347400e+07,1.158000e+06,3.683900e+07,-1.647000e+07,2.510200e+07,3.973800e+07,4.255000e+06,-2.899000e+06,1.143000e+06,-0.945389
7,1.280000e+05,3.942800e+07,-3.585000e+06,-9.735000e+06,9.150000e+05,1.959000e+06,-6.150000e+06,4.528500e+07,2.641000e+06,7.240000e+05,-6.820000e+05,-5.960000e+05,-0.246906
8,6.580000e+04,3.176360e+05,-9.894640e+05,-1.191474e+06,4.473900e+04,7.696710e+05,-2.020100e+05,3.101450e+05,8.970000e+04,4.410000e+04,6.799710e+05,2.170000e+04,-3.751067
9,1.452000e+03,1.403650e+05,-3.274000e+04,-5.357930e+05,1.101700e+04,4.960880e+05,-5.030530e+05,3.073030e+05,9.329430e+05,9.427700e+04,-4.368550e+05,-9.282500e+04,-3.817141


In [77]:
model_2016 = linear_model.LinearRegression(fit_intercept = True)
TotalNetAccruals = third_q['TOTALNETACCRUALS']
xxx = third_q[['ASSETSLAG', 'REVLAG', 'ARLAG' , 'REVCHANGE' , 'ARCHANGE', 'ROA',  'PropertyPlantAndEquipmentNet'  ]]
xxx = sm.add_constant(xxx)
model_2016 = sm.OLS(TotalNetAccruals, xxx).fit()
model_2016.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       TOTALNETACCRUALS   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                     116.2
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           7.06e-28
Time:                        13:01:58   Log-Likelihood:                -1120.7
No. Observations:                  56   AIC:                             2257.
Df Residuals:                      48   BIC:                             2274.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -1.795e+07   2.05e+07     -0.874      0.387   -5.93e+07    2.34e+07
ASSETSLAG                       -0.0014      0.038     -0.037      0.971      -0.077       0.075
REVLAG                          -0.2554      0.083     -3.072      0.003      -0.423      -0.088
ARLAG                            0.8177      0.573      1.426      0.160      -0.335       1.970
REVCHANGE                       -2.5193      0.217    -11.633      0.000      -2.955      -2.084
ARCHANGE                         7.5758      0.865      8.762      0.000       5.837       9.314
ROA                          -9.998e+05   4.46e+06     -0.224      0.824   -9.97e+06    7.97e+06
PropertyPlantAndEquipmentNet    -0.0293      0.116     -0.253      0.801      -0.262       0.204
==============================================================================
Omnibus:                       30.270   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              122.124
Skew:                          -1.283   Prob(JB):                     3.03e-27
Kurtosis:                       9.764   Cond. No.                     8.63e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.63e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We use a Regression Analysis to see how closley related independent variables are to dependent variables. 
"The result of pulling discretionary accrual amounts from the total accrual amount is a metric that 
reflects accruals that are due to management's choices alone; in other words, 
there appears to be no business reason for these accruals. So, discretionary accruals are 
a better proxy for earnings quality." We want to predict total net accruals(non discretionary) for companies based off certain independent variables. To estimate non discretionary accrual amounts, we use a time series analysis. We look at data
over a period of time. This model is created based off the 2016 period and the lagged variables are the prior period. 

An error term is a variable that shows a misrepresentation in the relationship between the independent and dependent variable. An error term in this case is represeneted by discretionary accruals. The higher the discretionary accruals the lower the quality of earnings. 

Inputs

Variables:

Dependent Variables: Total NET ACCRUALS

The Dependent variable is the variable that is being tested. We are creating our model based of 55 observations in 2016

Independent Variables:

Prior Year Assets

Prior Year Revenue

Change In Revenue

Change In Accounts REC

Return on Assets

As you can see from above we use lag variables to find the prior year assets

Results

Created a regression model to test total net accruals, based off 5 variables; Prior Year Assets, Prior Year Revenue, Change in Revenue, Change in REC, and change in assets. We want to use this model to predict total net accruals for firms and check the error rate to see managements decisions on accruals. The R squared value in this model is .944. R squared is the fraction of variation in your predicted variable in relation to your independent variable. That means that the closer the model is to 100%, explains that all the variablity is around the average. Based off these 55 observations, the model shows that 94.4 percent of the data is around the mean. As you see from, the predictions based off the model shows that the predicted total net accruals are closley related to the actual net accruals. Regression coefficients represents the mean change in the response variable for one unit of change in the independent variable. Change in AR has a coefficient of 7.5939, which  indicates that for every additional Recieavable you can expect Total NET ACCRUALS to increase by an average of 7.5939 dollars.

In [35]:
p = model_2016.predict()


In [36]:
df = pd.DataFrame(p , columns = ['Predicted TotalNetAccruals'])
df

,Predicted TotalNetAccruals
0,-7.537834e+07
1,3.202908e+07
2,-2.169655e+07
3,-1.999636e+09
4,-4.462744e+07
5,1.511382e+07
6,-8.492102e+06
7,-2.185740e+07
8,-1.641498e+07
9,-1.458390e+07


# TRAIN THE REGRESSION USING PRIOR DATA. GOODNESS OF FIT MEASURES VARIABLE COEFICIENTS for 2015. LAGGED VARIABLES ARE FOR 2014

In [39]:
num1 =  pd.read_csv('./2016q1/num.txt', sep = '\t', encoding="latin-1")
tag1 =  pd.read_csv('./2016q1/tag.txt', sep = '\t', encoding="latin-1")
pre1 =  pd.read_csv('./2016q1/pre.txt', sep = '\t', encoding="latin-1")
sub1 =  pd.read_csv('./2016q1/sub.txt', sep = '\t', encoding="latin-1")

### # 2015 Q4 Data 

In [40]:
firstmerge1 = pd.merge(pre1, sub1, how = "left", on=['adsh'])   #left merge

secondmerge1 = pd.merge(firstmerge1, tag1, how= "left", on =['tag', 'version'])

thirdmerge1 = pd.merge(secondmerge1, num1, how = "left", on =['adsh', 'tag', 'version'])
sec_table2015 = thirdmerge1[['cik','adsh' , 'afs' , 'qtrs' , 'fy' , 'fp', 'tag', 'version', 'value' , 'ddate']].sort_values(['cik'])

In [41]:
Regression_2015 =  sec_table2015[(sec_table2015.tag == 'NetIncomeLoss')
                        | (sec_table2015.tag == 'NetCashProvidedByUsedInOperatingActivities')
                        | (sec_table2015.tag == 'SalesRevenueNet')
                        | (sec_table2015.tag == 'Assets')
                        | (sec_table2015.tag == 'AccountsReceivableNetCurrent')
                        | (sec_table2015.tag == 'PropertyPlantAndEquipmentNet')]
Regression_2015

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
1110855,1750,0001104659-16-107128,1-LAF,1.0,2016.0,Q3,SalesRevenueNet,us-gaap/2015,4.028000e+08,20160229.0
1110854,1750,0001104659-16-107128,1-LAF,3.0,2016.0,Q3,SalesRevenueNet,us-gaap/2015,1.178500e+09,20150228.0
1110856,1750,0001104659-16-107128,1-LAF,3.0,2016.0,Q3,SalesRevenueNet,us-gaap/2015,1.204400e+09,20160229.0
741699,1750,0001104659-16-107128,1-LAF,0.0,2016.0,Q3,PropertyPlantAndEquipmentNet,us-gaap/2015,2.148000e+08,20150531.0
741700,1750,0001104659-16-107128,1-LAF,0.0,2016.0,Q3,PropertyPlantAndEquipmentNet,us-gaap/2015,2.224000e+08,20160229.0
...,...,...,...,...,...,...,...,...,...,...
734639,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,PropertyPlantAndEquipmentNet,us-gaap/2015,4.463200e+04,20141231.0
166784,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,Assets,us-gaap/2015,1.283508e+07,20141231.0
166785,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,Assets,us-gaap/2015,1.432558e+07,20151231.0
734640,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,PropertyPlantAndEquipmentNet,us-gaap/2015,2.187000e+04,20151231.0


In [42]:
Regression_2015.pivot_table(values='value', index=['tag', 'adsh','cik','ddate'], columns=['qtrs'])

qtrs                                                                        0.0    \
tag                          adsh                 cik     ddate                     
AccountsReceivableNetCurrent 0000002178-16-000064 2178    20141231.0  144434000.0   
                                                          20151231.0   71813000.0   
                             0000002488-16-000111 2488    20141231.0  818000000.0   
                                                          20151231.0  533000000.0   
                             0000003545-16-000130 3545    20150930.0    3137000.0   
...                                                                           ...   
SalesRevenueNet              0001391609-16-000415 1223533 20151231.0          NaN   
                             0001437749-16-028714 1322439 20150331.0          NaN   
                                                          20150630.0          NaN   
                                                          20150930.0          NaN   
                                                          20151231.0          NaN   

qtrs                                                                       1.0    \
tag                          adsh                 cik     ddate                    
AccountsReceivableNetCurrent 0000002178-16-000064 2178    20141231.0         NaN   
                                                          20151231.0         NaN   
                             0000002488-16-000111 2488    20141231.0         NaN   
                                                          20151231.0         NaN   
                             0000003545-16-000130 3545    20150930.0         NaN   
...                                                                          ...   
SalesRevenueNet              0001391609-16-000415 1223533 20151231.0         NaN   
                             0001437749-16-028714 1322439 20150331.0  26331166.0   
                                                          20150630.0  22657372.0   
                                                          20150930.0  29127482.0   
                                                          20151231.0  25740856.0   

qtrs                                                                  2.0    \
tag                          adsh                 cik     ddate               
AccountsReceivableNetCurrent 0000002178-16-000064 2178    20141231.0    NaN   
                                                          20151231.0    NaN   
                             0000002488-16-000111 2488    20141231.0    NaN   
                                                          20151231.0    NaN   
                             0000003545-16-000130 3545    20150930.0    NaN   
...                                                                     ...   
SalesRevenueNet              0001391609-16-000415 1223533 20151231.0    NaN   
                             0001437749-16-028714 1322439 20150331.0    NaN   
                                                          20150630.0    NaN   
                                                          20150930.0    NaN   
                                                          20151231.0    NaN   

qtrs                                                                  3.0    \
tag                          adsh                 cik     ddate               
AccountsReceivableNetCurrent 0000002178-16-000064 2178    20141231.0    NaN   
                                                          20151231.0    NaN   
                             0000002488-16-000111 2488    20141231.0    NaN   
                                                          20151231.0    NaN   
                             0000003545-16-000130 3545    20150930.0    NaN   
...                                                                     ...   
SalesRevenueNet              0001391609-16-000415 1223533 20151231.0    NaN   
                             0001437749-16-028714 1322439 2015

In [43]:
data_setQ42015 = Regression_2015[(Regression_2015.ddate == 20151231)
                   | (Regression_2015.ddate == 20141231)]
data_setQ42015

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
1105045,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.356000e+09,20141231.0
1105046,1800,0001047469-16-010246,1-LAF,4.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,2.024700e+10,20141231.0
1105050,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.188000e+09,20151231.0
1105051,1800,0001047469-16-010246,1-LAF,4.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,2.040500e+10,20151231.0
555664,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,9.050000e+08,20141231.0
...,...,...,...,...,...,...,...,...,...,...
734639,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,PropertyPlantAndEquipmentNet,us-gaap/2015,4.463200e+04,20141231.0
166784,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,Assets,us-gaap/2015,1.283508e+07,20141231.0
166785,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,Assets,us-gaap/2015,1.432558e+07,20151231.0
734640,1667489,0001548123-16-000469,5-SML,0.0,2015.0,FY,PropertyPlantAndEquipmentNet,us-gaap/2015,2.187000e+04,20151231.0


In [44]:
data_tableQ42015 = pd.pivot_table(data_setQ42015,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ42015['TOTALNETACCRUALS'] =  data_tableQ42015['NetIncomeLoss'] - data_tableQ42015['NetCashProvidedByUsedInOperatingActivities']
Q42015_Regression = data_tableQ42015.dropna(axis = 0)
Q42015_Regression['ASSETSLAG']=Q42015_Regression.groupby('cik').Assets.shift()
Q42015_Regression['REVLAG'] = Q42015_Regression.groupby('cik').SalesRevenueNet.shift()
Q42015_Regression['ARLAG'] = Q42015_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q42015_Regression['REVCHANGE'] = Q42015_Regression['SalesRevenueNet'] - Q42015_Regression['REVLAG']
Q42015_Regression['ARCHANGE'] = Q42015_Regression['AccountsReceivableNetCurrent'] - Q42015_Regression['ARLAG']
Q42015_Regression['ROA'] = Q42015_Regression['NetIncomeLoss'] / Q42015_Regression['Assets']
Regression_Q42015 = Q42015_Regression.dropna(axis = 0)
Regression_Q42015

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
1800,20151231.0,3.418000e+09,4.124700e+10,2.966000e+09,2.595000e+09,5.730000e+09,1.279650e+10,-371000000.0,4.120700e+10,1.280150e+10,3.586000e+09,-5.000000e+06,-168000000.0,0.062914
2488,20151231.0,5.330000e+08,3.109000e+09,-2.260000e+08,-3.810000e+08,1.880000e+08,3.991000e+09,-155000000.0,3.767000e+09,5.506000e+09,8.180000e+08,-1.515000e+09,-285000000.0,-0.122547
7536,20151231.0,6.161418e+09,1.302193e+10,6.550790e+08,3.281090e+08,7.001780e+08,1.501668e+10,-326970000.0,1.243530e+10,1.458278e+10,6.043850e+09,4.339045e+08,117568000.0,0.025197
9092,20151231.0,5.664300e+07,3.554800e+08,3.583100e+07,1.571050e+07,9.092000e+07,2.367340e+08,-20120500.0,3.411580e+08,2.270535e+08,5.396700e+07,9.680500e+06,2676000.0,0.044195
12208,20151231.0,3.914850e+08,3.711542e+09,1.862100e+08,8.129650e+07,4.376900e+08,1.295020e+09,-104913500.0,3.341278e+09,1.386622e+09,3.776400e+08,-9.160150e+07,13845000.0,0.021904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637655,20151231.0,6.305000e+07,3.315800e+08,2.691000e+07,8.300000e+06,4.589000e+07,3.483900e+08,-18610000.0,3.395000e+08,3.696750e+08,6.384000e+07,-2.128500e+07,-790000.0,0.025032
1637913,20151231.0,2.044000e+08,1.696300e+09,2.300000e+07,-1.165000e+07,2.218000e+08,7.429000e+08,-34650000.0,1.769800e+09,7.659000e+08,1.833000e+08,-2.300000e+07,21100000.0,-0.006868
1641614,20151231.0,5.253800e+07,2.803530e+08,4.392200e+07,1.428450e+07,5.211300e+07,2.338385e+08,-29637500.0,2.660100e+08,1.744355e+08,4.354800e+07,5.940300e+07,8990000.0,0.050952


Q3 2015

In [45]:
data_setQ32015 = Regression_2015[(Regression_2015.ddate == 20150930)
                   | (Regression_2015.ddate == 20140930)]
data_setQ32015

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
1105044,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.079000e+09,20140930.0
1105049,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.150000e+09,20150930.0
555663,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,5.380000e+08,20140930.0
555685,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,5.380000e+08,20140930.0
555690,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,5.800000e+08,20150930.0
...,...,...,...,...,...,...,...,...,...,...
589440,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,6.752169e+06,20140930.0
589434,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,-8.149132e+06,20150930.0
589418,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,6.752169e+06,20140930.0
589423,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,-8.149132e+06,20150930.0


In [46]:
data_tableQ32015 = pd.pivot_table(data_setQ32015,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ32015['TOTALNETACCRUALS'] =  data_tableQ32015['NetIncomeLoss'] - data_tableQ32015['NetCashProvidedByUsedInOperatingActivities']
Q32015_Regression = data_tableQ32015.dropna(axis = 0)
Q32015_Regression['ASSETSLAG']=Q32015_Regression.groupby('cik').Assets.shift()
Q32015_Regression['REVLAG'] = Q32015_Regression.groupby('cik').SalesRevenueNet.shift()
Q32015_Regression['ARLAG'] = Q32015_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q32015_Regression['REVCHANGE'] = Q32015_Regression['SalesRevenueNet'] - Q32015_Regression['REVLAG']
Q32015_Regression['ARCHANGE'] = Q32015_Regression['AccountsReceivableNetCurrent'] - Q32015_Regression['ARLAG']
Q32015_Regression['ROA'] = Q32015_Regression['NetIncomeLoss'] / Q32015_Regression['Assets']
Regression_Q32015 = Q32015_Regression.dropna(axis = 0)
Regression_Q32015

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
47307,20150930.0,1.101554e+06,3.866575e+06,1.997200e+04,9.646700e+04,3.752340e+05,3.823917e+06,76495.0,3.699657e+06,4.157276e+06,1.172268e+06,-3.333585e+05,-7.071400e+04,0.024949
53669,20150930.0,5.751000e+09,2.959500e+10,1.600000e+09,1.563000e+09,5.870000e+09,3.717900e+10,-37000000.0,3.278500e+10,3.874900e+10,5.871000e+09,-1.570000e+09,-1.200000e+08,0.052813
1117171,20150930.0,4.771958e+06,6.336723e+07,-2.037122e+06,1.587393e+07,2.227482e+07,1.390441e+07,17911048.0,4.396818e+07,1.230141e+08,1.013641e+06,-1.091097e+08,3.758317e+06,0.250507
1124941,20150930.0,1.816964e+08,7.007895e+08,4.373600e+07,2.491080e+07,4.109318e+07,1.006068e+09,-18825200.0,5.725608e+08,9.307620e+08,1.443208e+08,7.530560e+07,3.737556e+07,0.035547


# Q2 2015 Trained Data

In [47]:
data_setQ22015 = Regression_2015[(Regression_2015.ddate == 20150630)
                   | (Regression_2015.ddate == 20140630)]
data_setQ22015

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
1105048,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.170000e+09,20150630.0
1105043,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,5.057000e+09,20140630.0
555662,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,4.660000e+08,20140630.0
555667,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,7.840000e+08,20150630.0
555684,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,4.660000e+08,20140630.0
...,...,...,...,...,...,...,...,...,...,...
589433,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,-1.628023e+07,20150630.0
589439,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.999457e+06,20140630.0
589417,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.999457e+06,20140630.0
589422,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,-1.628023e+07,20150630.0


In [48]:
data_tableQ22015 = pd.pivot_table(data_setQ22015,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
data_tableQ22015['TOTALNETACCRUALS'] =  data_tableQ22015['NetIncomeLoss'] - data_tableQ22015['NetCashProvidedByUsedInOperatingActivities']
Q22015_Regression = data_tableQ22015.dropna(axis = 0)
Q22015_Regression['ASSETSLAG']=Q22015_Regression.groupby('cik').Assets.shift()
Q22015_Regression['REVLAG'] = Q22015_Regression.groupby('cik').SalesRevenueNet.shift()
Q22015_Regression['ARLAG'] = Q22015_Regression.groupby('cik').AccountsReceivableNetCurrent.shift()
Q22015_Regression['REVCHANGE'] = Q22015_Regression['SalesRevenueNet'] - Q22015_Regression['REVLAG']
Q22015_Regression['ARCHANGE'] = Q22015_Regression['AccountsReceivableNetCurrent'] - Q22015_Regression['ARLAG']
Q22015_Regression['ROA'] = Q22015_Regression['NetIncomeLoss'] / Q22015_Regression['Assets']
Regression_Q22015 = Q22015_Regression.dropna(axis = 0)
Regression_Q22015

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
1205181,20150630.0,1250.0,92633.0,-226481.0,-553932.0,25473.0,91129.5,-327451.0,38906.0,228005.0,22040.0,-136875.5,-20790.0,-5.979856


Q1 Trained data 2015

In [49]:
data_setQ12015 = Regression_2015[(Regression_2015.ddate == 20150331)
                   | (Regression_2015.ddate == 20140331)]
data_setQ12015

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
1105047,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,4.897000e+09,20150331.0
1105042,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,SalesRevenueNet,us-gaap/2015,4.755000e+09,20140331.0
555661,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,3.750000e+08,20140331.0
555666,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.292000e+09,20150331.0
555688,1800,0001047469-16-010246,1-LAF,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.292000e+09,20150331.0
...,...,...,...,...,...,...,...,...,...,...
589438,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.904691e+06,20140331.0
589427,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.904691e+06,20140331.0
589416,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,2.904691e+06,20140331.0
589421,1660719,0001140361-16-057746,2-ACC,1.0,2015.0,FY,NetIncomeLoss,us-gaap/2015,-2.735800e+04,20150331.0


In [50]:

first1 = pd.merge(Regression_Q22015, Regression_Q32015,
                 how = 'outer')
second1 = pd.merge(first1, Regression_Q42015,
                 how = 'outer')
second1

tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
0,1.250000e+03,9.263300e+04,-2.264810e+05,-5.539320e+05,2.547300e+04,9.112950e+04,-327451.0,3.890600e+04,2.280050e+05,2.204000e+04,-1.368755e+05,-2.079000e+04,-5.979856
1,1.101554e+06,3.866575e+06,1.997200e+04,9.646700e+04,3.752340e+05,3.823917e+06,76495.0,3.699657e+06,4.157276e+06,1.172268e+06,-3.333585e+05,-7.071400e+04,0.024949
2,5.751000e+09,2.959500e+10,1.600000e+09,1.563000e+09,5.870000e+09,3.717900e+10,-37000000.0,3.278500e+10,3.874900e+10,5.871000e+09,-1.570000e+09,-1.200000e+08,0.052813
3,4.771958e+06,6.336723e+07,-2.037122e+06,1.587393e+07,2.227482e+07,1.390441e+07,17911048.0,4.396818e+07,1.230141e+08,1.013641e+06,-1.091097e+08,3.758317e+06,0.250507
4,1.816964e+08,7.007895e+08,4.373600e+07,2.491080e+07,4.109318e+07,1.006068e+09,-18825200.0,5.725608e+08,9.307620e+08,1.443208e+08,7.530560e+07,3.737556e+07,0.035547
...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,6.305000e+07,3.315800e+08,2.691000e+07,8.300000e+06,4.589000e+07,3.483900e+08,-18610000.0,3.395000e+08,3.696750e+08,6.384000e+07,-2.128500e+07,-7.900000e+05,0.025032
406,2.044000e+08,1.696300e+09,2.300000e+07,-1.165000e+07,2.218000e+08,7.429000e+08,-34650000.0,1.769800e+09,7.659000e+08,1.833000e+08,-2.300000e+07,2.110000e+07,-0.006868
407,5.253800e+07,2.803530e+08,4.392200e+07,1.428450e+07,5.211300e+07,2.338385e+08,-29637500.0,2.660100e+08,1.744355e+08,4.354800e+07,5.940300e+07,8.990000e+06,0.050952
408,4.839000e+08,3.309400e+09,2.136000e+08,5.430000e+07,4.324000e+08,1.500600e+09,-159300000.0,4.028100e+09,1.746050e+09,5.919000e+08,-2.454500e+08,-1.080000e+08,0.016408


In [51]:
model_2015 = linear_model.LinearRegression(fit_intercept = True)
ACCRUALS = second1['TOTALNETACCRUALS']
ddd = second1[['ASSETSLAG', 'REVLAG', 'ARLAG' , 'REVCHANGE' , 'ARCHANGE', 'ROA' ]]
ddd = sm.add_constant(ddd)
model_2015 = sm.OLS(ACCRUALS, ddd).fit()
model_2015.summary()

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       TOTALNETACCRUALS   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     131.8
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           8.50e-92
Time:                        12:29:12   Log-Likelihood:                -8898.5
No. Observations:                 410   AIC:                         1.781e+04
Df Residuals:                     403   BIC:                         1.784e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.455e+07   3.56e+07      1.251      0.212   -2.54e+07    1.15e+08
ASSETSLAG     -0.0712      0.005    -15.573      0.000      -0.080      -0.062
REVLAG        -0.0115      0.009     -1.290      0.198      -0.029       0.006
ARLAG         -0.0997      0.064     -1.567      0.118      -0.225       0.025
REVCHANGE     -0.4725      0.043    -11.074      0.000      -0.556      -0.389
ARCHANGE       1.7139      0.264      6.491      0.000       1.195       2.233
ROA         4.016e+07   5.54e+07      0.724      0.469   -6.88e+07    1.49e+08
==============================================================================
Omnibus:                      473.612   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            82476.717
Skew:                          -4.829   Prob(JB):                         0.00
Kurtosis:                      71.809   Cond. No.                     2.18e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.18e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [52]:
second1.head()

tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
0,1.250000e+03,9.263300e+04,-2.264810e+05,-5.539320e+05,2.547300e+04,9.112950e+04,-327451.0,3.890600e+04,2.280050e+05,2.204000e+04,-1.368755e+05,-2.079000e+04,-5.979856
1,1.101554e+06,3.866575e+06,1.997200e+04,9.646700e+04,3.752340e+05,3.823917e+06,76495.0,3.699657e+06,4.157276e+06,1.172268e+06,-3.333585e+05,-7.071400e+04,0.024949
2,5.751000e+09,2.959500e+10,1.600000e+09,1.563000e+09,5.870000e+09,3.717900e+10,-37000000.0,3.278500e+10,3.874900e+10,5.871000e+09,-1.570000e+09,-1.200000e+08,0.052813
3,4.771958e+06,6.336723e+07,-2.037122e+06,1.587393e+07,2.227482e+07,1.390441e+07,17911048.0,4.396818e+07,1.230141e+08,1.013641e+06,-1.091097e+08,3.758317e+06,0.250507
4,1.816964e+08,7.007895e+08,4.373600e+07,2.491080e+07,4.109318e+07,1.006068e+09,-18825200.0,5.725608e+08,9.307620e+08,1.443208e+08,7.530560e+07,3.737556e+07,0.035547


In [53]:
pp = model_2015.predict()
df_pp = pd.DataFrame(pp , columns = ['Trained Predicted TotalNetAccruals'])
df_pp


,Trained Predicted TotalNetAccruals
0,-1.956121e+08
1,4.515605e+07
2,-2.784391e+09
3,1.079548e+08
4,8.540150e+06
...,...
405,1.944689e+07
406,-6.184099e+07
407,8.632275e+06
408,-3.899247e+08


In [54]:
df_pp.head()

,Trained Predicted TotalNetAccruals
0,-1.956121e+08
1,4.515605e+07
2,-2.784391e+09
3,1.079548e+08
4,8.540150e+06


I created the regression model based off the 2016 period and the 2015 prior period. We trained this model to find a goodness of fit and coefficients to show the relationships between total accruals to the 5 variables used in both models.

2016 actual model analysis:
We use a Regression Analysis to see how closley related independent variables are to dependent variables. 
"The result of pulling discretionary accrual amounts from the total accrual amount is a metric that 
reflects accruals that are due to management's choices alone; in other words, 
there appears to be no business reason for these accruals. So, discretionary accruals are 
a better proxy for earnings quality." We want to predict total net accruals(non discretionary) for companies based off certain independent variables. To estimate non discretionary accrual amounts, we use a time series analysis. We look at data
over a period of time. This model is created based off the 2016 period and the lagged variables are the prior period. 
An error term is a variable that shows a misrepresentation in the relationship between the independent and dependent variable. An error term in this case is represeneted by discretionary accruals. The higher the discretionary accruals the lower the quality of earnings. 
Inputs
Variables:
Dependent Variables: Total NET ACCRUALS
The Dependent variable is the variable that is being tested. We are creating our model based of 55 observations in 2016
Independent Variables:
Prior Year Assets
Prior Year Revenue
PRIOR YEAR AR
Change In Revenue
Change In Accounts REC
Return on Assets

As you can see from above we use lag variables to find the prior year assets

Results

Created a regression model to test total net accruals, based off 5 variables; Prior Year Assets, Prior Year Revenue, Change in Revenue, Change in REC, and change in assets. We want to use this model to predict total net accruals for firms and check the error rate to see managements decisions on accruals. The R squared value in this model is .944. R squared is the fraction of variation in your predicted variable in relation to your independent variable. That means that the closer the model is to 100%, explains that all the variablity is around the average. Based off these 55 observations, the model shows that 94.4 percent of the data is around the mean. As you see from, the predictions based off the model shows that the predicted total net accruals are closley related to the actual net accruals. Regression coefficients represents the mean change in the response variable for one unit of change in the independent variable. Change in AR has a coefficient of 7.5939, which  indicates that for every additional Recieavable you can expect Total NET ACCRUALS to increase by an average of 7.5939 dollars.

COMPARISION:

We used the prior period to to test the regression model. I imported data from 2016 to get more observations for my regression model. I thought it would be better to train the actual model with over 410 observations. 

Goodness of Fit:

A model fits the data if the models observed values and the models predicted values. As you can see from the r squared, of .662, that the explained variation/total variation is approx 66.2 percent. The actual model had 55 observations, which could be an explanation behind the 94.4% r squared value. The closer to 100%, the less variation in the model. As you can see from the predicted observations from above, I compared the first 4 of the predicted observations to the actual observations to see the realibility of the model. The first 4 predicted net accrual amount are: 

0 -1.956121e+08

1	4.515605e+07

2	-2.784391e+09

3	1.079548e+08

4	8.540150e+06

The 4 actual obervations for net accruals are :

1)327451

2)76495

3)37000000

4)17911048

You can see there are variations between the predicted observations and actual observations, but thats due to the fact that the model only explains 64.4% of the variations.

Variable Coefficients 

2015 Trained Model:

Prior Year Assets: -.0712

Prior Year Revenue: -0115

AR LAG: -.0997

Change In Revenue:  -.4725

Change In Accounts REC :  1.7139

Return on Assets: .0000004316

Constant: 4.455e+07



2016 Actual Model:

Prior Year Assets: -.0068

Prior Year Revenue -0.2517

Prior Year AR: .8629

Change In Revenue: -2.5053 

Change In Accounts REC: 7.5939

Return on Assets: -1.085e+06

Constant: -1.895e+07

The first observation that I found is the return on assets coefficient value. The value is so small that it shows a little to no relationship between return on assets and total net accruals. To get a better r squared value I would reccomend dropping the return on assets coefficient. 

The second observation that I found is the signifcance of the p values in both models. The higher the p value, represents the changes in the predictor is not responsive. The common alpha value is .05 and if it is greater than .05 indicates that the variable is not statistically significant. A predictor that has a low p value represents greater responsivness between changes in the tested variable. As you can see from the model, there were significant changes in p values in prior year assets, and prior year revenue from the actual model to the trained model. Prior year assets represents a significant variable in the trained model and not a significant variable in the actual model. Return on assets shows no responsivness which means that we could ommit this variable for a better model. Change in Revenue and AR showed perfect responsivness to total accruals and should be used predict the model. 



PVALUES ACTUAL MODEL:

Prior Year Assets: .827  > .05

Prior Year Revenue 0.003 <.05

Prior Year AR: .116 > .05

Change In Revenue: .000  < .05

Change In Accounts REC: 0.00  < .05

Return on Assets: .806 > .05

PVALUES TRAINED MODEL:

Prior Year Assets: .000  < .05

Prior Year Revenue : .198 > .05

Prior Year AR: .118 > .05

Change In Revenue: .0000 < .05

Change In Accounts REC: .0000 <.05

Return on Assets: .469  > .05

Improvements to further test the model:

Based off the p values and the change in r squared I decided to make some changes to the model for further testing. I will test total net accruals based off the variables that have low p values. The new regression model is going to ommit return on assets, the constant, and prior year AR. I decided to keep the coefficient for prior year revenue because of the low p value in the trained model. The model is going to be used to test future net accruals in the 2017 quarter 3 periods. We are going to look at if the predicted values are too high, too low, or equal to the actual observations. 


### test model on 2017 data q3

In [56]:
num4 =  pd.read_csv('./2017q3/num.txt', sep = '\t', encoding="latin-1")
tag4 =  pd.read_csv('./2017q3/tag.txt', sep = '\t', encoding="latin-1")
pre4 =  pd.read_csv('./2017q3/pre.txt', sep = '\t', encoding="latin-1")
sub4 =  pd.read_csv('./2017q3/sub.txt', sep = '\t', encoding="latin-1")

In [57]:
firstmerge4 = pd.merge(pre4, sub4, how = "left", on=['adsh'])   #left merge

secondmerge4 = pd.merge(firstmerge4, tag4, how= "inner", on =['tag', 'version'])

thirdmerge4 = pd.merge(secondmerge4, num4, how = "inner", on =['adsh', 'tag', 'version'])
sec_data2017 = thirdmerge4[['cik','adsh' , 'afs' , 'qtrs' , 'fy' , 'fp', 'tag', 'version', 'value' , 'ddate']].sort_values(['cik'])
sec_data2017

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
158530,1750,0001047469-17-004528,1-LAF,0,2017.0,FY,DeferredTaxLiabilitiesNoncurrent,us-gaap/2016,3.720000e+07,20170531
73518,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,7.900000e+06,20160831
73517,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,3.120000e+07,20160531
73516,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,1.510000e+07,20170831
73515,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,CashAndCashEquivalentsAtCarryingValue,us-gaap/2016,1.030000e+07,20170531
...,...,...,...,...,...,...,...,...,...,...
1504508,1712041,0001615774-17-004906,5-SML,2,2017.0,Q2,RepaymentsOfNotesPayable,us-gaap/2017,2.544000e+03,20170630
1504509,1712041,0001615774-17-004906,5-SML,4,2017.0,Q2,RepaymentsOfNotesPayable,us-gaap/2017,5.477200e+04,20161231
1504510,1712041,0001615774-17-004906,5-SML,4,2017.0,Q2,RepaymentsOfNotesPayable,us-gaap/2017,NaN,20151231
982100,1712041,0001615774-17-004906,5-SML,2,2017.0,Q2,GeneralAndAdministrativeExpense,us-gaap/2017,4.706800e+04,20160630


In [58]:
Regression_2017 =  sec_data2017[(sec_data2017.tag == 'NetIncomeLoss')
                        | (sec_data2017.tag == 'NetCashProvidedByUsedInOperatingActivities')
                        | (sec_data2017.tag == 'SalesRevenueNet')
                        | (sec_data2017.tag == 'Assets')
                        | (sec_data2017.tag == 'AccountsReceivableNetCurrent')
                        | (sec_data2017.tag == 'PropertyPlantAndEquipmentNet')]
Regression_2017

,cik,adsh,afs,qtrs,fy,fp,tag,version,value,ddate
44008,1750,0001047469-17-004528,1-LAF,0,2017.0,FY,Assets,us-gaap/2016,1.454100e+09,20150531
43430,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,Assets,us-gaap/2016,1.531700e+09,20170831
44010,1750,0001047469-17-004528,1-LAF,0,2017.0,FY,Assets,us-gaap/2016,1.504100e+09,20170531
44009,1750,0001047469-17-004528,1-LAF,0,2017.0,FY,Assets,us-gaap/2016,1.456000e+09,20160531
43429,1750,0001104659-17-058003,1-LAF,0,2018.0,Q1,Assets,us-gaap/2016,1.504100e+09,20170531
...,...,...,...,...,...,...,...,...,...,...
1243734,1712041,0001615774-17-004906,5-SML,4,2017.0,Q2,NetIncomeLoss,us-gaap/2017,-3.384840e+05,20161231
1243733,1712041,0001615774-17-004906,5-SML,2,2017.0,Q2,NetIncomeLoss,us-gaap/2017,-1.338076e+06,20170630
1486352,1712041,0001615774-17-004906,5-SML,0,2017.0,Q2,PropertyPlantAndEquipmentNet,us-gaap/2017,1.942000e+03,20170630
1486351,1712041,0001615774-17-004906,5-SML,0,2017.0,Q2,PropertyPlantAndEquipmentNet,us-gaap/2017,2.910000e+03,20151231


In [59]:
Q3 = Regression_2017[(Regression_2017.ddate == 20170630)
                   | (Regression_2017.ddate == 20160630)]


In [60]:
Q317 = pd.pivot_table(Q3,
                             values = 'value',
                             index = ['cik','ddate'],
                             columns = 'tag',
                             dropna = True)
Q317

Q317['TOTALNETACCRUALS'] =  Q317['NetIncomeLoss'] - Q317['NetCashProvidedByUsedInOperatingActivities']
Q317



Q317.dropna(axis=0)

Q317['ASSETSLAG']=Q317.groupby('cik').Assets.shift()
Q317['REVLAG'] = Q317.groupby('cik').SalesRevenueNet.shift()
Q317['ARLAG'] = Q317.groupby('cik').AccountsReceivableNetCurrent.shift()
Q317['REVCHANGE'] = Q317['SalesRevenueNet'] - Q317['REVLAG']
Q317['ARCHANGE'] = Q317['AccountsReceivableNetCurrent'] - Q317['ARLAG']
Q317['ROA'] = Q317['NetIncomeLoss'] / Q317['Assets']
Regression_Q317 = Q317.dropna(axis = 0)
Regression_Q317

,tag,AccountsReceivableNetCurrent,Assets,NetCashProvidedByUsedInOperatingActivities,NetIncomeLoss,PropertyPlantAndEquipmentNet,SalesRevenueNet,TOTALNETACCRUALS,ASSETSLAG,REVLAG,ARLAG,REVCHANGE,ARCHANGE,ROA
cik,ddate,,,,,,,,,,,,,
8670,20170630,1.703600e+09,3.718000e+10,2.125900e+09,9.996000e+08,7.799000e+08,8.518100e+09,-1.126300e+09,4.367000e+10,8.234000e+09,1.742800e+09,2.841000e+08,-39200000.0,0.026885
14195,20170630,2.300110e+08,1.450979e+09,9.034400e+07,5.665000e+07,3.648800e+08,1.786103e+09,-3.369400e+07,1.456667e+09,1.808778e+09,1.916780e+08,-2.267500e+07,38333000.0,0.039043
37472,20170630,4.236200e+07,2.700450e+08,2.638800e+07,1.490350e+07,7.066100e+07,2.930990e+08,-1.148450e+07,2.468960e+08,3.114350e+08,4.461800e+07,-1.833600e+07,-2256000.0,0.055189
50471,20170630,4.009127e+06,4.591248e+07,2.257138e+06,3.777532e+06,2.115277e+06,1.893926e+07,1.520394e+06,3.858989e+07,1.401069e+07,3.048774e+06,4.928570e+06,960353.0,0.082277
55242,20170630,3.804250e+08,2.415496e+09,1.922020e+08,3.689050e+07,7.443880e+08,1.311696e+09,-1.553115e+08,2.362783e+09,1.309830e+09,3.709160e+08,1.866500e+06,9509000.0,0.015272
80424,20170630,4.594000e+09,1.204060e+11,1.275300e+10,8.770500e+09,1.989300e+10,4.056850e+10,-3.982500e+09,1.271360e+11,4.070050e+10,4.373000e+09,-1.320000e+08,221000000.0,0.072841
89800,20170630,2.377874e+09,2.071740e+10,5.861330e+08,4.386870e+08,1.889917e+09,5.116510e+09,-1.474460e+08,6.665677e+09,4.506537e+09,1.473078e+09,6.099735e+08,904796000.0,0.021175
92679,20170630,4.534700e+07,2.171430e+08,3.146800e+07,1.502000e+06,3.445500e+07,2.509740e+08,-2.996600e+07,2.459980e+08,2.631645e+08,4.675900e+07,-1.219050e+07,-1412000.0,0.006917
100548,20170630,6.924600e+07,2.904532e+09,2.366010e+08,6.539200e+07,2.174609e+09,4.573685e+08,-1.712090e+08,2.697301e+09,4.002975e+08,5.930100e+07,5.707100e+07,9945000.0,0.022514


In [61]:
Regression_Q317['Accruals'] = -.0712 * Regression_Q317['ASSETSLAG'] + -.2517 * Regression_Q317['REVLAG'] + -2.5053 * Regression_Q317['REVCHANGE'] + 7.5939 * Regression_Q317['ARCHANGE']

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
model_2017 = linear_model.LinearRegression(fit_intercept = True)
RUALS = second1['TOTALNETACCRUALS']
hhh = second1[['ASSETSLAG', 'REVLAG', 'REVCHANGE' , 'ARCHANGE']]
hhh = sm.add_constant(hhh)
model_2017 = sm.OLS(RUALS, hhh).fit()
model_2017.summary()

/Users/varunganti/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       TOTALNETACCRUALS   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     196.5
Date:                Tue, 25 Feb 2020   Prob (F-statistic):           1.82e-93
Time:                        12:34:19   Log-Likelihood:                -8900.0
No. Observations:                 410   AIC:                         1.781e+04
Df Residuals:                     405   BIC:                         1.783e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.157e+07   3.45e+07      0.915      0.361   -3.63e+07    9.94e+07
ASSETSLAG     -0.0733      0.004    -16.818      0.000      -0.082      -0.065
REVLAG        -0.0202      0.007     -2.911      0.004      -0.034      -0.007
REVCHANGE     -0.4771      0.043    -11.192      0.000      -0.561      -0.393
ARCHANGE       1.7626      0.263      6.708      0.000       1.246       2.279
==============================================================================
Omnibus:                      496.924   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            93528.344
Skew:                          -5.255   Prob(JB):                         0.00
Kurtosis:                      76.242   Cond. No.                     1.33e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.33e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [63]:
ppp = model_2015.predict()
df_ppp = pd.DataFrame(ppp , columns = ['Tested Predicted TotalNetAccruals'])
df_ppp

,Tested Predicted TotalNetAccruals
0,-1.956121e+08
1,4.515605e+07
2,-2.784391e+09
3,1.079548e+08
4,8.540150e+06
...,...
405,1.944689e+07
406,-6.184099e+07
407,8.632275e+06
408,-3.899247e+08


In [66]:
df_append = Regression_Q317.loc[:, ['TOTALNETACCRUALS' , 'Accruals']]
df_append
df_append['Variance'] = df_append['TOTALNETACCRUALS'] - df_append['Accruals']
df_append

,tag,TOTALNETACCRUALS,Accruals,Variance
cik,ddate,,,
8670,20170630,-1.126300e+09,-6.191238e+09,5.064938e+09
14195,20170630,-3.369400e+07,-2.110795e+08,1.773855e+08
37472,20170630,-1.148450e+07,-6.716184e+07,5.567734e+07
50471,20170630,1.520394e+06,-1.132881e+07,1.284921e+07
55242,20170630,-1.553115e+08,-4.303801e+08,2.750686e+08
80424,20170630,-3.982500e+09,-1.728745e+10,1.330495e+10
89800,20170630,-1.474460e+08,3.733872e+09,-3.881318e+09
92679,20170630,-2.996600e+07,-6.393529e+07,3.396929e+07
100548,20170630,-1.712090e+08,-3.602614e+08,1.890524e+08


In [67]:
high_amount = df_append[(df_append['Variance'] > 0)]
high_amount

,tag,TOTALNETACCRUALS,Accruals,Variance
cik,ddate,,,
8670,20170630,-1.126300e+09,-6.191238e+09,5.064938e+09
14195,20170630,-3.369400e+07,-2.110795e+08,1.773855e+08
37472,20170630,-1.148450e+07,-6.716184e+07,5.567734e+07
50471,20170630,1.520394e+06,-1.132881e+07,1.284921e+07
55242,20170630,-1.553115e+08,-4.303801e+08,2.750686e+08
80424,20170630,-3.982500e+09,-1.728745e+10,1.330495e+10
92679,20170630,-2.996600e+07,-6.393529e+07,3.396929e+07
100548,20170630,-1.712090e+08,-3.602614e+08,1.890524e+08
102037,20170630,1.206560e+08,-2.757670e+08,3.964230e+08


In [68]:
low_amount = df_append[(df_append['Variance'] < 0)]
low_amount

,tag,TOTALNETACCRUALS,Accruals,Variance
cik,ddate,,,
89800,20170630,-147446000.0,3.733872e+09,-3.881318e+09
310354,20170630,-33101000.0,-2.662546e+07,-6.475542e+06
722830,20170630,-40980328.5,-4.229805e+06,-3.675052e+07
788920,20170630,1849000.0,3.188645e+06,-1.339645e+06
811212,20170630,-21880000.0,-9.019142e+06,-1.286086e+07
811532,20170630,-28161500.0,2.126675e+08,-2.408290e+08
939930,20170630,193748000.0,3.266461e+08,-1.328981e+08
1078271,20170630,-67800000.0,-3.551893e+07,-3.228107e+07
1336917,20170630,49324500.0,1.430321e+08,-9.370759e+07


In [69]:
exact_amount = df_append[(df_append['Variance'] == 0)]
exact_amount

,tag,TOTALNETACCRUALS,Accruals,Variance
cik,ddate,,,


In [70]:
print('The total amount of companies are' , len(df_append))

The total amount of companies are 54


In [72]:
df_conclusion = df_append.describe()
df_conclusion

tag,TOTALNETACCRUALS,Accruals,Variance
count,5.400000e+01,5.400000e+01,5.400000e+01
mean,-1.944957e+08,-7.513773e+08,5.568816e+08
std,5.977192e+08,2.590300e+09,2.038848e+09
min,-3.982500e+09,-1.728745e+10,-3.881318e+09
25%,-1.616029e+08,-4.866569e+08,1.392966e+07
50%,-2.250625e+07,-1.206777e+08,1.352061e+08
75%,1.397052e+05,-2.101279e+07,3.955414e+08
max,2.190165e+08,3.733872e+09,1.330495e+10


In [73]:
df_append['EXPECTEDAVERAGE'] = df_append['Variance']/ df_append['TOTALNETACCRUALS']  * 100
df_append

,tag,TOTALNETACCRUALS,Accruals,Variance,EXPECTEDAVERAGE
cik,ddate,,,,
8670,20170630,-1.126300e+09,-6.191238e+09,5.064938e+09,-449.697098
14195,20170630,-3.369400e+07,-2.110795e+08,1.773855e+08,-526.460102
37472,20170630,-1.148450e+07,-6.716184e+07,5.567734e+07,-484.804234
50471,20170630,1.520394e+06,-1.132881e+07,1.284921e+07,845.123535
55242,20170630,-1.553115e+08,-4.303801e+08,2.750686e+08,-177.107689
80424,20170630,-3.982500e+09,-1.728745e+10,1.330495e+10,-334.085312
89800,20170630,-1.474460e+08,3.733872e+09,-3.881318e+09,2632.365862
92679,20170630,-2.996600e+07,-6.393529e+07,3.396929e+07,-113.359439
100548,20170630,-1.712090e+08,-3.602614e+08,1.890524e+08,-110.421971


In [74]:
high_variance = df_append[(df_append['EXPECTEDAVERAGE'] < -40)]
print('Percentages lower than -40% of the data' , len(high_variance))

Percentages lower than -40% of the data 36


In [75]:
print('The amount of observations' , len(df_append))

The amount of observations 54


I took the percentage of the varaince over the actual amounts to compare and see how close the percentage change is to 0. I took -40% as a threshold to see how many percentages fall below 40%. I found that 36 of the 54 of the observations fell below this threshold, which shows that over 70% of the data shows a high amount of variance. When there is a high level of variance in a model in the model, it shows that new variables should be implemented to better test total accruals. I also ran a regression model on the 2017 Q3 data to see how responsive variables are relative to tested variables. R squared value of .660 shows the model has a good amount of variance, which shows that our actual tested variable has deviations from any predictions made in the future.

In conclusion, the jones model is a way for companies to predict non discretionary accruals across firms using time series data. Being able to extract industry wide financial data, and predict future companys amounts based off multiple regression. We want to be able to find relationships between relationships, and create models to help conclude on hypothesis and test variables that are in interest of management. If I were to work for a big bank, I could use financial data for individual customers to predict uncollectibles and write off credit based off prior data. Although, our model has a lot of variance, that comes to show that we have to continue to find new variables to test to find the least amount of variance. We can look at p values that are lower than .05 which will show the level of responsivness for certain variables. We saw, on average, that the change in revenues and change in accounts recievables have the biggest effect on the change in accruals. We see in many fraud cases, that trends in recievables and sales can be examined to see any overstatements in receivables and revenue. In the next model, I want to add non cash financing activites, like depreciation, as variables into the model to see if there is any relationship to accruals. Many companies have different ways to identify tags in the database, so it is useful to extract data from the sec taxonomy to see the most frequent tags used in the industry. This will allow us analysts to find more relationships in accruals across the industry

In [2]:
def plot_predictions(model_name, y_train_pred ,y_train_actual):
    plt.scatter(y_train_actual, y_train_pred, c = "blue", marker = "s", label = "Training data")
    plt.title("Regression Sales Price")
    plt.xlabel("$\hat{y}$ = Actual SalePrice($)")
    plt.ylabel("$y$ = Predicted SalePrice($)")
    plt.legend(loc = "upper left")
    plt.plot([y_train_actual.min(), y_train_actual.max()], [y_train_actual.min(), y_train_actual.max()], c='red')
    plt.tight_layout()